# Intruduction to NLP fundamentals in TensorFlow
NLP has the goal of deriving information out of natural language (could be sequences of text or speech).

Another common term for NLP problems is sequence to sequence problems (seq2seq).

## Check for the GPU

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-537df743-0a99-09e4-59d6-6c6462520ce4)


## Get helper functions

In [2]:
# Download the helper functions
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2023-03-22 19:12:31--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-03-22 19:12:31 (86.3 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset
The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labeled as disaster or not disaster).

See the orignal source here: https://www.kaggle.com/competitions/nlp-getting-started

In [3]:
# Download the data
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data('nlp_getting_started.zip')

--2023-03-22 19:12:39--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.132.128, 74.125.201.128, 74.125.202.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.132.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.009s  

2023-03-22 19:12:39 (66.5 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Become one with the data - visualize a text dataset

To visualize our text sample, we first need to read them in. One way to do so would be to use Python.

But we can get visual straight away.

So another way to do this is to use Pandas.

TF help: https://www.tensorflow.org/tutorials/load_data/text

In [4]:
import pandas as pd

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df['text'][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [6]:
train_df['text'][1]

'Forest fire near La Ronge Sask. Canada'

In [7]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, 
                                    random_state=42)

train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
# What does the test dataset look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
# How many examples are there?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

If dataset is imbalanced: https://www.tensorflow.org/tutorials/structured_data/imbalanced_data

In [10]:
# How many total samples
len(train_df), len(test_df)

(7613, 3263)

In [11]:
# Let's visualize some random training examples
import random 
random_index = random.randint(0, len(train_df)-5) #create random indexes not higher then random number of samples

for row in train_df_shuffled[['text', 'target']][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text: \n{text}\n")
  print("---\n")

Target: 0 (not real disaster)
Text: 
http://t.co/iNkuv5DNTX #auction #shoes Retro 5 fire red http://t.co/1cvEGTIZOG

---

Target: 1 (real disaster)
Text: 
Islamic State claims suicide bombing at Saudi Arabian mosque: Ten members of emergency service... http://t.co/mpOaEFQl6k via @josephjett

---

Target: 1 (real disaster)
Text: 
The sign-up is open for the FALLING FOR DANGER release day blast and review tour. Sign-up here:... http://t.co/hbdo22nqPZ

---

Target: 1 (real disaster)
Text: 
@TheBlackshag @dannyoneil too toxic...cancer....disease...hazardous waste...noxious...

---

Target: 0 (not real disaster)
Text: 
@michelleellle ?? shut up freshman its past ur curfew. u need some sleep!! u spend too much of ur time watching tv instead of going outside ??

---



### Split data into training and validation sets

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
# Use train_test_split to split the training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'],
                                                                            train_df_shuffled['target'],
                                                                            test_size=0.1,
                                                                            random_state=42)

In [14]:
# Check the lenghts
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [15]:
# Check some top samples
train_sentences[:10], train_labels[:10]

(5921    @mogacola @zamtriossu i screamed after hitting...
 3904              Imagine getting flattened by Kurt Zouma
 2804    @Gurmeetramrahim #MSGDoing111WelfareWorks Gree...
 3718    @shakjn @C7 @Magnums im shaking in fear he's g...
 1667    Somehow find you and I collide http://t.co/Ee8...
 4435    @EvaHanderek @MarleyKnysh great times until th...
 2544                     destroy the free fandom honestly
 7223    Weapons stolen from National Guard Armory in N...
 4265    @wfaaweather Pete when will the heat wave pass...
 6568    Patient-reported outcomes in long-term survivo...
 Name: text, dtype: object, 5921    0
 3904    0
 2804    1
 3718    0
 1667    0
 4435    1
 2544    1
 7223    0
 4265    1
 6568    1
 Name: target, dtype: int64)

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a model is to convert your text into numbers.

There are a few ways to do this, namely:
* Tokenization - direct mapping of token (a token could be a word or a character) to number
* Embeddings - create a matrix of feature vector for each token (the size of the feature vector can be defined and this embeddings can be learned)

### Text vectorizaton - tokenization

In [16]:
train_sentences[:5]

5921    @mogacola @zamtriossu i screamed after hitting...
3904              Imagine getting flattened by Kurt Zouma
2804    @Gurmeetramrahim #MSGDoing111WelfareWorks Gree...
3718    @shakjn @C7 @Magnums im shaking in fear he's g...
1667    Somehow find you and I collide http://t.co/Ee8...
Name: text, dtype: object

In [17]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [18]:
# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (automaticall add <OOV>)
                                    standardize='lower_and_strip_punctuation',
                                    split='whitespace',
                                    ngrams=None, # create groups of n-words?
                                    output_mode='int', # how to map tokens to numbers
                                    output_sequence_length=None, # how long you want your sequences to be
                                    #pad_to_max_tokens=True
                                    )

In [19]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [20]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words in vocabulary
max_length = 15 # max length our sequences will be (e.g. for many words will a model see)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length)

In [21]:
# Fit the text vextorizer to the training set
text_vectorizer.adapt(train_sentences)

In [22]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [23]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
      \n\nVectorized version: ")
text_vectorizer([random_sentence])

Original text:
 @BenKin97 @Mili_5499 remember when u were up like 4-0 and blew it in one game? U probs don't because it was before the kings won the cup      

Vectorized version: 


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  1,   1, 594,  45, 142,  67,  27,  25, 354,   7, 555,  15,   4,
         61, 397]])>

In [24]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words in training data
top_5_words = words_in_vocab[:5] # most 5 common words
bottom_5_words = words_in_vocab[-5:]

print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words are: {top_5_words}")
print(f"Top 5 least common words are: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words are: ['', '[UNK]', 'the', 'a', 'in']
Top 5 least common words are: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Create and Embedding using and Embedding Layer
To make our embedding we're going to use TensorFlow's Embedding Layer.

Parameters we care the most are:
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example: a value of 100 would mean each token gets represented by a vector 100 long
* `input_length` = length of sequences being passes to the embedding layer

In [25]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # output shape
                             input_length=max_length #how long in each input
                             )

embedding

In [26]:
# Get another random sentence
random_sentence = random.choice(train_sentences)
print(f"Original text: {random_sentence}")
print(f"Embedded version: ")

# Embed the random sentence (turn it into the dense vector of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text: Well so much for outdoor postering @calgaryfringe #mothernature #hailstorm #rain #yycfringe #KILLHARD
Embedded version: 


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-3.79151106e-03, -8.47990438e-03, -1.23155005e-02, ...,
          3.46574225e-02, -1.87554844e-02,  1.74659491e-03],
        [-3.44484672e-02, -4.17119041e-02,  2.59278305e-02, ...,
          1.71197541e-02,  3.59235294e-02, -2.20036861e-02],
        [ 2.88693048e-02, -1.76449791e-02,  2.90600993e-02, ...,
          4.68134619e-02,  4.97085191e-02, -1.25393979e-02],
        ...,
        [-9.18686390e-04,  3.52784991e-06,  2.76698917e-03, ...,
          9.31248814e-03, -2.86061764e-02, -2.28958130e-02],
        [-9.18686390e-04,  3.52784991e-06,  2.76698917e-03, ...,
          9.31248814e-03, -2.86061764e-02, -2.28958130e-02],
        [-9.18686390e-04,  3.52784991e-06,  2.76698917e-03, ...,
          9.31248814e-03, -2.86061764e-02, -2.28958130e-02]]],
      dtype=float32)>

In [27]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-3.79151106e-03, -8.47990438e-03, -1.23155005e-02,  4.72400226e-02,
         3.74482907e-02, -8.03745911e-03, -2.94770133e-02, -3.35872062e-02,
        -4.91266511e-02, -3.55492830e-02,  1.18766427e-02, -1.88810583e-02,
         6.85504824e-03, -4.30230610e-02,  9.04492289e-03,  2.25400664e-02,
         2.21837275e-02,  1.01812854e-02, -4.79272977e-02, -2.63975020e-02,
        -3.93969417e-02, -4.19388898e-02, -9.73069668e-03,  3.24008204e-02,
        -2.44721025e-03, -7.47387484e-03, -4.20062654e-02, -4.82944138e-02,
        -8.57419893e-03,  4.37496938e-02,  6.78550452e-04,  1.85780190e-02,
         1.52916200e-02,  7.89176300e-03,  4.58773226e-03, -2.99355630e-02,
        -9.32936743e-03, -1.90593842e-02, -1.31447203e-02,  8.02614540e-03,
         3.92881669e-02, -4.09743898e-02,  2.34518759e-02, -4.54181693e-02,
         2.56788172e-02,  3.31434943e-02,  2.98328511e-02,  3.62524651e-02,
         6.94258139e-03,  3.10071222e-0

## Modelling a text dataset (running a bunch of experiments)

Now we've got a way to turn our text sequences into numbers, it's time to start building a series of modelling experiments.

We'll start with a baseline and move on from there.

* Model 0: Naive Bayes (baseline), this is from Sklearn ML Map
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM Model (RNN)
* Model 3: GRU Model (RNN)
* Model 4: Bidirectional-LSTM (RNN)
* Model 5: 1D Convolutional Neural Network (CNN)
* Model 6: TensorFlow Hub Pretrained feature Extractor (using transfer learning for NLP)
* Model 7: Same as model 6, but with 10% training data

### Model 0: getting a baseline

As with all machine learning modelling experiments, it's important to create a baseline model so you've got a benchmark for future experiments to build upon. 

To create our baseline, we'll use Sklearn's Multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers.

> **NOTE:** It's common practice to use non-DL algorythms as a baseline model because of their speed and then later using DL to see if you can improve upon them.


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()), # convert words to numbers using TF-IDF
    ('clf', MultinomialNB()) # model the text
])

# Fit the pipeline to the data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [29]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [30]:
# Make predictions 
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments

We could evaluate all of our model's predictions with different metrics every time. However this may bu cumbersom and will surely take some time.

Let's create one to compare our model's predcitions with truth labels using the following metrics:
* Accuracy
* Precision
* Recall
* F1-score

In [31]:
# Function to evaluate: accuracy, recall, precision, f1-score

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculate model accuracy, recall, precision and f1-score of a binary classification model.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100

  # Calculate precision, recall and f1-score using 'weighted' metrics
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

  model_results = {'accuracy': model_accuracy,
                   'precision': model_precision,
                   'recall': model_recall,
                   'f1': model_f1}

  return model_results

In [32]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)

baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: A simple dense model

In [33]:
# Create a tensorboard callback (need to create new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [34]:
# Build model with Functional API
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of numberized inputs
x = layers.GlobalAveragePooling1D()(x) # condense the feature vector for each token to each vector
outputs = layers.Dense(1, activation='sigmoid')(x) # create the output layer, want binary output so sigmoid

model_1 = tf.keras.Model(inputs, outputs, name='model_1_dense')

In [35]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [36]:
# Compile model
model_1.compile(loss='binary_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])

In [37]:
# Fit the model
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name='model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20230322-191248
Epoch 1/5
215/215 [==============================] - 17s 53ms/step - loss: 0.6125 - accuracy: 0.6930 - val_loss: 0.5352 - val_accuracy: 0.7625
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.4432 - accuracy: 0.8180 - val_loss: 0.4675 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.3488 - accuracy: 0.8606 - val_loss: 0.4568 - val_accuracy: 0.7913
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2854 - accuracy: 0.8891 - val_loss: 0.4645 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.2381 - accuracy: 0.9111 - val_loss: 0.4809 - val_accuracy: 0.7808


In [38]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 4ms/step - loss: 0.4809 - accuracy: 0.7808


[0.4808727204799652, 0.7808399200439453]

In [39]:
# Make some predictions and evaluate those
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


array([[0.32402468],
       [0.7305316 ],
       [0.99711347],
       [0.14024135],
       [0.09704939],
       [0.93447226],
       [0.90929466],
       [0.99207   ],
       [0.96043587],
       [0.31167376]], dtype=float32)

In [40]:
# Single prediction
model_1_pred_probs[1]

array([0.7305316], dtype=float32)

In [41]:
# Convert model prediction probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [42]:
# Calculate our model_1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 78.08398950131233,
 'precision': 0.7848699684611963,
 'recall': 0.7808398950131233,
 'f1': 0.7779792730264676}

In [43]:
import numpy as np
np.array(list(model_1_results.values()))> np.array(list(baseline_results.values()))

array([False, False, False, False])

## Visualizing learned embeddings

In [44]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [45]:
# Model_1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [46]:
# Get the weight matrix of embedding layer
# (these are the numerical representation of each token in our training data
# which have been learned for 5 epochs)
embed_weights = model_1.get_layer('embedding').get_weights()[0]
embed_weights

array([[-0.0165341 , -0.01156985,  0.01828641, ..., -0.00566531,
        -0.0164663 , -0.03809671],
       [-0.06167642,  0.03693729, -0.02772405, ..., -0.04792374,
        -0.01465564, -0.05094279],
       [-0.04264189, -0.03407058, -0.02361921, ...,  0.0345754 ,
        -0.02575861,  0.02730376],
       ...,
       [-0.02529265,  0.00396367, -0.01636482, ...,  0.02642414,
         0.02134185, -0.01224426],
       [-0.02269838, -0.06684919,  0.01318339, ..., -0.06196566,
         0.0051993 , -0.00876852],
       [-0.0658977 , -0.02980096,  0.08537647, ..., -0.05522868,
         0.11287592, -0.01355842]], dtype=float32)

In [47]:
embed_weights.shape # same size as vocab size and embedding_dim (output_dim of our embedding layer)

(10000, 128)

Now we've got the embedding matrix, our model has learned to represent our tokens. Let's see how we can visualize it.

To do so, TensorFlow has a handy tool called projector: http://projector.tensorflow.org/

And TensorFlow also has an incredible guide on word embeddings themselves.

In [48]:
# Code straight from TensorFlow documentation
# Create embedding files
import io

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [49]:
# Download files from Colab to import them to Projector
# Code straight from TensorFlow documentation
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Recurrent Neural Networks (RNN's)

RNN's are useful for sequential data.

The premise of a recurrent neural network is to use the representation of a precious input to aid the representation of a later input.

### Model 2: LSTM

LSTM = Long short term memory (one of the most popular RNN cells).

Our structure of an RNN typically looks like this:

```
Input (text) -> Tokenize -> Embedding -> Layers (RNNs/Dense) -> Output (label probability)



In [50]:
# Create an LSTM model

from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
#print(x.shape)
#x = layers.LSTM(units=64, return_sequences=True)(x) # when you're stacking RNN cells together, you need to return_sequences=True
#print(x.shape)
x = layers.LSTM(64)(x)
#print(x.shape)
#x = layers.Dense(64, activation='relu')(x)
#print(x.shape)
outputs = layers.Dense(1, activation='sigmoid')(x)

model_2 = tf.keras.Model(inputs, outputs, name='model_2_LSTM')

In [51]:
# Get a summary
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [52]:
# Compile the model
model_2.compile(loss='binary_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])

In [53]:
# Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                                     'model_2_LSTM')])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230322-191333
Epoch 1/5
215/215 [==============================] - 14s 48ms/step - loss: 0.2267 - accuracy: 0.9215 - val_loss: 0.5301 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1575 - accuracy: 0.9419 - val_loss: 0.6456 - val_accuracy: 0.7677
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1286 - accuracy: 0.9514 - val_loss: 0.6919 - val_accuracy: 0.7835
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1084 - accuracy: 0.9594 - val_loss: 0.8301 - val_accuracy: 0.7769
Epoch 5/5
215/215 [==============================] - 2s 7ms/step - loss: 0.0894 - accuracy: 0.9650 - val_loss: 0.8586 - val_accuracy: 0.7743


In [54]:
# Make predictions with LSTM model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 1s 4ms/step


array([[1.6782122e-02],
       [6.0469967e-01],
       [9.9960083e-01],
       [1.1015288e-01],
       [4.0242885e-04],
       [9.9543238e-01],
       [8.7401116e-01],
       [9.9967003e-01],
       [9.9945503e-01],
       [6.4194614e-01]], dtype=float32)

In [55]:
# Convert model_2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [56]:
# Calculate model_2 results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 77.42782152230971,
 'precision': 0.7752857985262857,
 'recall': 0.7742782152230971,
 'f1': 0.7725974162749719}

In [57]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 3: GRU
Another popular and effective RNN component is the GRU layer (or Gated Recurrent Unit).

The GRU cell has similar features to LSTM cell but has less parameters.

In [58]:
# Build an RNN using the GRU cell
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
#x = layers.GRU(64, return_sequences=True)(x) # return_sequences=True if stacking layers
#x = layers.LSTM(64, return_sequences=True)(x)
#x = layers.GRU(64)(x)
#x = layers.Dense(64, activation='relu')(x)
#x = layers.GlobalAveragePooling1D()(x)
x = layers.GRU(64)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model_3 = tf.keras.Model(inputs, outputs, name='model_3_GRU')

In [59]:
# Check the summary
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [60]:
# Compile the model
model_3.compile(loss='binary_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])

In [61]:
# Fit the model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     'model_3_GRU')])

Saving TensorBoard log files to: model_logs/model_3_GRU/20230322-191356
Epoch 1/5
215/215 [==============================] - 11s 37ms/step - loss: 0.1594 - accuracy: 0.9356 - val_loss: 0.7415 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0802 - accuracy: 0.9695 - val_loss: 0.7875 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 2s 7ms/step - loss: 0.0704 - accuracy: 0.9734 - val_loss: 0.8739 - val_accuracy: 0.7730
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0616 - accuracy: 0.9759 - val_loss: 1.0506 - val_accuracy: 0.7743
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0508 - accuracy: 0.9787 - val_loss: 1.2976 - val_accuracy: 0.7690


In [62]:
# Make predictions with GRU model
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


array([[1.9183676e-04],
       [8.0372691e-01],
       [9.9981517e-01],
       [5.0000466e-02],
       [4.9625487e-05],
       [9.9962246e-01],
       [7.8384084e-01],
       [9.9995506e-01],
       [9.9987686e-01],
       [9.2680556e-01]], dtype=float32)

In [63]:
# Convert model 3 pred probs to labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [64]:
# Calculate model 3 results
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_preds)
model_3_results

{'accuracy': 76.9028871391076,
 'precision': 0.7731540113429877,
 'recall': 0.7690288713910761,
 'f1': 0.7657803921853192}

### Model 4: Bidirectional RNN

Normal RNNs go from left to right (just the same way you read an English sentence). However, bidirectional RNN goes from right to left as well as from left to right.

In [65]:
# Create a bidirectional RNN model 4
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model_4 = tf.keras.Model(inputs, outputs, name='model_4_bidirectional')

In [66]:
# Check the summary
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [67]:
# Compile the model
model_4.compile(loss='binary_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])

In [68]:
# Fit the model
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     'model_4_bidirectional')])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20230322-191416
Epoch 1/5
215/215 [==============================] - 14s 44ms/step - loss: 0.1049 - accuracy: 0.9660 - val_loss: 0.9885 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0537 - accuracy: 0.9777 - val_loss: 1.1291 - val_accuracy: 0.7730
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0451 - accuracy: 0.9803 - val_loss: 1.2522 - val_accuracy: 0.7664
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0411 - accuracy: 0.9801 - val_loss: 1.2665 - val_accuracy: 0.7598
Epoch 5/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0439 - accuracy: 0.9812 - val_loss: 1.1781 - val_accuracy: 0.7730


In [69]:
# Make predicitons
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 1s 3ms/step


array([[2.7189220e-03],
       [7.2863185e-01],
       [9.9988735e-01],
       [1.4180492e-01],
       [1.0002956e-04],
       [9.9536419e-01],
       [6.9474512e-01],
       [9.9993443e-01],
       [9.9987376e-01],
       [7.4568182e-01]], dtype=float32)

In [70]:
# Convert predictions
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [71]:
# Calculate results
model_4_results = calculate_results(y_true=val_labels,
                                    y_pred=model_4_preds)
model_4_results

{'accuracy': 77.29658792650919,
 'precision': 0.7766832892495897,
 'recall': 0.7729658792650919,
 'f1': 0.7700024804405922}

## Convolutional Neural Networks for Text (and other sequences)

We've used CNNs for images, but images are typically 1D (H x W), however our text data is 1D.

Previoulsy we've used `Conv2D()` for image data but now we're going to use `Conv1D()` as data is 1-dimensional.

The typical structure of a Conv1D model for sequences (in our case, text):
```
Inputs (text) -> Tokenization -> Embedding -> Layer(s) (typically Conv1D + pooling) -> Outputs
```

### Model 5: 1D Convolutional Neural Network

In [72]:
# Test out our embedding layer, Conv1D layer and MaxPooling
from tensorflow.keras import layers

embedding_test = embedding(text_vectorizer(['this is a test sequence'])) # turn target sequence into embedding
conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5, # also referred as an ngram=5 (looks at 5 words at a time)
                        strides=1, # default
                        activation='relu',
                        padding='same') # default = valid, output is smaller than input, same -> input is same shape as output
conv_1d_output = conv_1d(embedding_test) # pass embedding through Conv1D layer
max_pool = layers.GlobalMaxPooling1D()
max_pool_output = max_pool(conv_1d_output) # equaivalent to 'get the most important feature/feature with the highest value'

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 15, 32]), TensorShape([1, 32]))

In [73]:
#embedding_test

In [74]:
#conv_1d_output

In [75]:
#max_pool_output

In [89]:
# Create 1-dimensional convolutional layer to model sequences
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=64,
                  kernel_size=5,
                  activation='relu',
                  padding='valid',
                  strides=1)(x)
x = layers.GlobalMaxPool1D()(x)
# x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model_5 = tf.keras.Model(inputs, outputs, name='model_5_Conv1D')

# Compile Conv1D
model_5.compile(loss='binary_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])

# Summary
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_3 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_3 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_7 (Dense)             (None, 1)              

In [90]:
# Fit the model
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     'model_5_Conv1D')])

Saving TensorBoard log files to: model_logs/model_5_Conv1D/20230322-192738
Epoch 1/5
215/215 [==============================] - 10s 40ms/step - loss: 0.0809 - accuracy: 0.9768 - val_loss: 1.0679 - val_accuracy: 0.7572
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0486 - accuracy: 0.9813 - val_loss: 1.1331 - val_accuracy: 0.7546
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0448 - accuracy: 0.9807 - val_loss: 1.2385 - val_accuracy: 0.7480
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0440 - accuracy: 0.9825 - val_loss: 1.2286 - val_accuracy: 0.7493
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0411 - accuracy: 0.9815 - val_loss: 1.2310 - val_accuracy: 0.7428


In [91]:
# Make predictions
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[3.3602845e-02],
       [9.6805435e-01],
       [9.9997509e-01],
       [8.5860863e-02],
       [2.1820324e-06],
       [9.9792457e-01],
       [9.7088796e-01],
       [9.9999166e-01],
       [9.9999976e-01],
       [9.5681852e-01]], dtype=float32)

In [92]:
# Convert predictions
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [93]:
# Calculate the results
model_5_results = calculate_results(y_true=val_labels,
                                    y_pred=model_5_preds)
model_5_results

{'accuracy': 74.2782152230971,
 'precision': 0.7423147810749008,
 'recall': 0.7427821522309711,
 'f1': 0.7423631256619239}

## Model 6 - TensorFlow Hub Pretrained Sentence Encoder

Now we've built some of our own models, let's try and use transfer learning for NLP, specifically using Universal Sentence Encoder (or USE for short).

In [97]:
 import tensorflow_hub as hub
 embed = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')
 embed_samples = embed([sample_sentence, 'When you call the universal sentence encoder on a sentence, it turns it into numbers.'])

 print(embed_samples[0][:50])

tf.Tensor(
[-0.01157028  0.0248591   0.02878048 -0.012715    0.03971538  0.0882776
  0.02680984  0.05589836 -0.0106873  -0.00597291  0.00639323 -0.01819518
  0.00030813  0.09105888  0.05874644 -0.03180628  0.01512474 -0.05162929
  0.00991367 -0.06865347 -0.04209306  0.02678981  0.03011006  0.00321069
 -0.00337973 -0.04787357  0.0226672  -0.00985925 -0.04063613 -0.01292092
 -0.04666384  0.05630299 -0.03949255  0.00517686  0.02495829 -0.0701444
  0.02871508  0.04947684 -0.00633979 -0.08960192  0.02807118 -0.00808364
 -0.01360602  0.0599865  -0.10361787 -0.05195374  0.00232954 -0.02332531
 -0.03758105  0.03327728], shape=(50,), dtype=float32)


In [96]:
embed_samples

<tf.Tensor: shape=(2, 512), dtype=float32, numpy=
array([[-0.01157028,  0.0248591 ,  0.02878048, ..., -0.00186124,
         0.02315826, -0.01485021],
       [ 0.03596687, -0.08579469, -0.0115274 , ..., -0.03414334,
         0.02816025, -0.00878945]], dtype=float32)>

In [98]:
embed_samples.shape

TensorShape([2, 512])

In [105]:
# Create a Keras Layer using the USE pretrained layer from TensorFlow Hub
sentence_encoder_layer = hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4', 
                                       input_shape=[], 
                                       dtype=tf.string,
                                       trainable=False,
                                       name='USE')

In [135]:
# Create model using Sequential API
model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(128, activation='relu'),
    #layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid', name='output_layer')
], name='model_6_USE')

# Compile the model
model_6.compile(loss='binary_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])

# Summary
model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_16 (Dense)            (None, 128)               65664     
                                                                 
 output_layer (Dense)        (None, 1)                 129       
                                                                 
Total params: 256,863,617
Trainable params: 65,793
Non-trainable params: 256,797,824
_________________________________________________________________


In [136]:
# Train the classifier on top of USE pretrained embeddings
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     'model_6_USE')])

Saving TensorBoard log files to: model_logs/model_6_USE/20230322-201447
Epoch 1/5
215/215 [==============================] - 6s 15ms/step - loss: 0.4840 - accuracy: 0.7900 - val_loss: 0.4506 - val_accuracy: 0.8058
Epoch 2/5
215/215 [==============================] - 4s 17ms/step - loss: 0.4093 - accuracy: 0.8209 - val_loss: 0.4381 - val_accuracy: 0.8150
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3940 - accuracy: 0.8286 - val_loss: 0.4306 - val_accuracy: 0.8136
Epoch 4/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3844 - accuracy: 0.8318 - val_loss: 0.4231 - val_accuracy: 0.8176
Epoch 5/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3738 - accuracy: 0.8358 - val_loss: 0.4268 - val_accuracy: 0.8123


In [137]:
# Make predicitons
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:10]

24/24 [==============================] - 1s 10ms/step


array([[0.16229738],
       [0.810685  ],
       [0.99247277],
       [0.21266091],
       [0.69216317],
       [0.73202914],
       [0.9824677 ],
       [0.9843313 ],
       [0.9436467 ],
       [0.07944041]], dtype=float32)

In [138]:
# Convert predicition probs
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [139]:
# Calcaulate results
model_6_results = calculate_results(y_true=val_labels,
                                    y_pred=model_6_preds)
model_6_results

{'accuracy': 81.23359580052494,
 'precision': 0.8167328378435911,
 'recall': 0.8123359580052494,
 'f1': 0.8101294251786122}

In [140]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [141]:
len(train_sentences)

6851

## Model 7: TF Hub Pretrained USE, but with 10% of training data

Transfer learning really helps if you do not have a large dataset. 

To see how our model performs on a smaller dataset, let's replicate `model_6` except we'll train it on 10% of the data.

In [174]:
## NOTE: Making data splits like below creates data leakage (model_7 trained on 10% of the data, outperforms model_6 trained on 100% of the data)
## DO NOT MAKE DATA SPLITS WHICH LEAK DATA FROM VALIDATION/TEST SETS INTO TRAINING SET
'''
# Create subset of 10% of the training data
train_10_percent = train_df_shuffled[['text', 'target']].sample(frac=0.1, random_state=42)

train_sentences_10_percent = train_10_percent['text'].to_list()
train_labels_10_percent = train_10_percent['target'].to_list()

len(train_10_percent), len(train_10_percent)
'''

"\n# Create subset of 10% of the training data\ntrain_10_percent = train_df_shuffled[['text', 'target']].sample(frac=0.1, random_state=42)\n\ntrain_sentences_10_percent = train_10_percent['text'].to_list()\ntrain_labels_10_percent = train_10_percent['target'].to_list()\n\nlen(train_10_percent), len(train_10_percent)\n"

In [178]:
# Making a better dataset split (no data leakage)
train_10_percent_split = int(0.1*len(train_sentences))
train_sentences_10_percent = train_sentences[:train_10_percent_split]
train_labels_10_percent = train_labels[:train_10_percent_split]
len(train_sentences_10_percent), len(train_labels_10_percent)

(685, 685)

In [180]:
# Check the number of each label in updated subset
pd.Series(np.array(train_labels_10_percent)).value_counts()

0    406
1    279
dtype: int64

In [182]:
# Check the number of targets in our subset data
# train_labels_10_percent['target'].value_counts()

KeyError: ignored

In [145]:
train_df_shuffled['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [183]:
# Recreating model 6
model_7 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid', name='output_layer')
], name='model_7_USE_10_percent')

# Compile the model
model_7.compile(loss='binary_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])

# Summary
model_7.summary()

Model: "model_7_USE_10_percent"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_25 (Dense)            (None, 128)               65664     
                                                                 
 output_layer (Dense)        (None, 1)                 129       
                                                                 
Total params: 256,863,617
Trainable params: 65,793
Non-trainable params: 256,797,824
_________________________________________________________________


In [184]:
# Fit the model on 10% of the dataset
model_7_history = model_7.fit(train_sentences_10_percent,
                              train_labels_10_percent,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     'model_7_USE_10_percent_correct_split')])

Saving TensorBoard log files to: model_logs/model_7_USE_10_percent_correct_split/20230322-211900
Epoch 1/5
22/22 [==============================] - 4s 60ms/step - loss: 0.6588 - accuracy: 0.7372 - val_loss: 0.6279 - val_accuracy: 0.7349
Epoch 2/5
22/22 [==============================] - 1s 27ms/step - loss: 0.5628 - accuracy: 0.8073 - val_loss: 0.5609 - val_accuracy: 0.7612
Epoch 3/5
22/22 [==============================] - 0s 23ms/step - loss: 0.4773 - accuracy: 0.8131 - val_loss: 0.5058 - val_accuracy: 0.7795
Epoch 4/5
22/22 [==============================] - 0s 23ms/step - loss: 0.4224 - accuracy: 0.8321 - val_loss: 0.4875 - val_accuracy: 0.7808
Epoch 5/5
22/22 [==============================] - 1s 27ms/step - loss: 0.3847 - accuracy: 0.8423 - val_loss: 0.4836 - val_accuracy: 0.7782


**Alternative method - cloning a model**<br>
To recreate a model the same as previous, you can use `tf.keras.model.clone_model()` method.

In [ ]:
# Create a model
# model_7 = tf.keras.model.clone_model(model_6, name)

# Compile the model
# model_7.compile(loss='binary_crossentropy',
#               optimizer='Adam',
#               metrics=['accuracy'])

# Summary
# model_7.summary()

In [185]:
# Make predictions
model_7_pred_probs = model_7.predict(val_sentences)
model_7_pred_probs[:10]

24/24 [==============================] - 1s 10ms/step


array([[0.15781969],
       [0.57956934],
       [0.95138556],
       [0.35519707],
       [0.5605337 ],
       [0.7180959 ],
       [0.934155  ],
       [0.8597891 ],
       [0.8919702 ],
       [0.13854818]], dtype=float32)

In [186]:
# Convert probs
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [187]:
# Get the results
model_7_results = calculate_results(y_true=val_labels,
                                    y_pred=model_7_preds)
model_7_results

{'accuracy': 77.82152230971128,
 'precision': 0.7779972615729265,
 'recall': 0.7782152230971129,
 'f1': 0.7775081589477405}

In [173]:
model_6_results

{'accuracy': 81.23359580052494,
 'precision': 0.8167328378435911,
 'recall': 0.8123359580052494,
 'f1': 0.8101294251786122}

> **NOTE**: Be *very* careful when creating training/val/test splits that you do not leak daa across the datasets, otherwise your model evaluation metrics will be wrong. If something looks too good to be true (a model trained on 10% of data outperforming the same model trained on 100% of the data), trust your gut and go back through to find where the error may lie.